# Conversion of sensors data into meaningful table


# Import necessary libraries

In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy.stats.mstats import winsorize

In [3]:
# Creat directory
directory = 'C:\\Users\\Mohamed\\OneDrive\\Documents\\GitHub\\Balance_PhyPhox\\Data'
# Making an empty datafram
results_df =pd.DataFrame(columns=['Folder_name','File_name','Condition','Axis','Average','Minimum','Maximum', 'Standard_deviation', 'coefficient of variation'])
# iterate over folders in the directory
for foldername in os.listdir(directory):
    f = os.path.join(directory, foldername)
    Folder_name= os.path.basename(f).split('/')[-1]
    # iterate over files in the folder
    for filename in os.listdir(f):
        file = os.path.join(f, filename)
        file_name= os.path.basename(file).split('/')[-1]
        file_name = file_name.split('.')[0]
        # Path of Raw Data 
        DATA_PATH = file
        # import DATA after having the LABLE (Frist Columen is Y) columnes 2 and 3 are the neuomerical data of predectors form thim Y calculated)
        df = pd.read_csv(DATA_PATH) 
        # Classifying conditions 
        df_C1= df.loc[(df['Time (s)']>=3) & (df['Time (s)']< 27)]
        df_C2= df.loc[(df['Time (s)']>=33) & (df['Time (s)']< 57)]
        df_C3= df.loc[(df['Time (s)']>=63) & (df['Time (s)']< 87)]
        df_C4= df.loc[(df['Time (s)']>=93) & (df['Time (s)']< 117)]
        Condition_list = [df_C1, df_C2, df_C3,df_C4] 
        for idx1, condition in enumerate(Condition_list):
            # Removing to outliers
            '''lower_limit = 0.0
            for j in df:
                condition[j] = winsorize(condition[j],limits = [lower_limit, 0.05])'''
            condition_name = f"Condition {idx1 + 1}" # condition.iloc[1, [0]]
            df_wins = condition
            # Classfying Axis
            X_Dimension=df_wins.iloc[:,[1]]
            Y_Dimension=df_wins.iloc[:,[2]]
            Z_Dimension=df_wins.iloc[:,[3]]
            Dimension_list = [X_Dimension, Y_Dimension, Z_Dimension]
            for idx2, Dimension in enumerate(Dimension_list):
                Axis = f"Dimension {idx2 + 1}" # Dimension
                q1, q2, q3 = np.quantile(Dimension, [0.25, 0.5, 0.75])
                up, down = q3 + 1.5*(q3-q1), q1-1.5*(q3-q1)
                ##Limits = frange(up, down-0.00000001)### + int(down)
                inliers = Dimension[(Dimension >= down) & (Dimension <= up)]
                #Dmension_NO_Outlier = Dimension[np.abs(Dimension.iloc[:,[0]]-Dimension.iloc[:,[0]].mean())<=(3*Dimension.iloc[:,[0]].std())]
                Dimension = inliers # Dmension_NO_Outlier
                Dimension_Average=  Dimension.mean().values
                Dimension_Min=  Dimension.min().values
                Dimension_Max=  Dimension.max().values
                Dimension_STD=  Dimension.std().values
                cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
                Dimension_CV=Dimension.apply(cv).values
                # Insert results in dataframe
                '''Folder_name','File_name',,,'''
                results_df = results_df.append({'Folder_name':Folder_name,'File_name': file_name,'Condition': condition_name, 'Axis':Axis,'Average': Dimension_Average,'Minimum': Dimension_Min,'Maximum': Dimension_Max, 
                'Standard_deviation': Dimension_STD,  'coefficient of variation': Dimension_CV}, ignore_index=True)


C:\Users\Mohamed\AppData\Local\Temp\ipykernel_15180\773999350.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Folder_name':Folder_name,'File_name': file_name,'Condition': condition_name, 'Axis':Axis,'Average': Dimension_Average,'Minimum': Dimension_Min,'Maximum': Dimension_Max,
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_15180\773999350.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Folder_name':Folder_name,'File_name': file_name,'Condition': condition_name, 'Axis':Axis,'Average': Dimension_Average,'Minimum': Dimension_Min,'Maximum': Dimension_Max,
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_15180\773999350.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [4]:
results_df

,Folder_name,File_name,Condition,Axis,Average,Minimum,Maximum,Standard_deviation,coefficient of variation
0,001,Accelerometer,Condition 1,Dimension 1,[-1.892861409369448],[-2.165504456],[-1.622333765],[0.09662197898645178],[-5.104545874736735]
1,001,Accelerometer,Condition 1,Dimension 2,[9.423788395895476],[9.353543282],[9.494720459],[0.02714347138703849],[0.2880314184352952]
2,001,Accelerometer,Condition 1,Dimension 3,[0.8466252664169738],[0.4977070987],[1.194018483],[0.13050474939230897],[15.414700525608186]
3,001,Accelerometer,Condition 2,Dimension 1,[-1.8433309623544154],[-2.000399828],[-1.686939955],[0.05973808719348317],[-3.2407683923011863]
4,001,Accelerometer,Condition 2,Dimension 2,[9.45145128756998],[9.38465023],[9.518648148],[0.02406724772748433],[0.2546407635739098]
...,...,...,...,...,...,...,...,...,...
139,003,Magnetometer,Condition 3,Dimension 2,[-29.811290224778617],[-32.8125],[-26.81250191],[1.118738932852722],[-3.752735706564106]
140,003,Magnetometer,Condition 3,Dimension 3,[-32.85467073820181],[-34.61250305],[-31.05000114],[0.6458476514191337],[-1.9657711884118003]
141,003,Magnetometer,Condition 4,Dimension 1,[14.367477135829143],[12.05625057],[16.36875153],[0.8663814750772265],[6.030157326067169]
142,003,Magnetometer,Condition 4,Dimension 2,[-31.21775281062826],[-33.43125153],[-28.95000076],[0.8380707002303074],[-2.684596502875061]


# Export data

In [5]:
#----------------------------------------------------
# Export the results to the excel
#----------------------------------------------------
EXPORT_PATH = r'C:\Users\Mohamed\OneDrive\Documents\GitHub\Balance_PhyPhox\Results\results_df.xlsx'
results_df.to_excel(EXPORT_PATH, index=False)